# Correction on an exact solution

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from utils import *
config = read_json_file()
method = config["method"]
method = method.lower()
if method == "fem":
    from solver_fem import *
    ClassSolver = FEMSolver
elif method == "phifem":
    from solver_phifem import *
    ClassSolver = PhiFemSolver
else:
    raise ValueError("Method not recognized")

geometry = config["geometry"]
if geometry == "circle":
    from data_circle import *
    int_a = 0.0
    int_b = 1.0
elif geometry == "square":
    from data_square import *
    int_a = -0.5
    int_b = 1.5

homogeneous = config["homogeneous"]
S = 0.5
if homogeneous:
    p = 0.
else:
    p = 1.

run = False

bash: /home/lecourtier/anaconda3/envs/env_tensorflow/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/lecourtier/anaconda3/envs/env_tensorflow/lib/libtinfo.so.6: no version information available (required by bash)


In [4]:
import os

result_dir = "results/"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

if homogeneous:
    result_dir += "homogeneous/"
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
else:
    result_dir += "non_homogeneous/"
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
result_file = result_dir+"nvert_100/"+"tab_errors_"+method+"_"+geometry
if not homogeneous:
    result_file+="_nh"
print(result_file)

results/homogeneous/nvert_100/tab_errors_fem_circle


In [5]:
import numpy as np
import pandas as pd
import dataframe_image as dfi
from dolfin import *

# noinspection PyUnresolvedReferences
from dolfin import parameters
import matplotlib.pyplot as plt

parameters["ghost_mode"] = "shared_facet"
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["optimize"] = True
parameters["allow_extrapolation"] = True
parameters["form_compiler"]["representation"] = "uflacs"
parameters["form_compiler"]["quadrature_degree"] = 10

## Data

In [6]:
def test_omega():
    mask = omega_mask(64)
    plt.imshow(mask)
    plt.colorbar()
    plt.show()

# test_omega()

In [7]:
def plot_data(F,U):
    mask = omega_mask(64)
    plt.figure(figsize=(9,3))

    plt.subplot(1,2,1)
    plt.imshow(mask * F, origin="lower")
    plt.title("F")
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(mask * U, origin="lower")
    plt.title("U_ex")
    plt.colorbar()

    plt.tight_layout()
    plt.show()

def test():
    f = 1
    XXYY = get_vert_coord(nb_vert=64)
    F = np.reshape(call_F(np,XXYY,S,f,p),[64,64])
    U = np.reshape(call_Y_true(np,XXYY,S,f,p),[64,64])
    plot_data(F,U)

# test()

## Results with differents frequencies

In [8]:
freqs = [1,2,3,4]
params = [[S,f,p] for f in freqs]

nb_vert = 32
solver = ClassSolver(nb_cell=nb_vert-1, params=params)

if run:
    tab_errors = []
    for i in range(len(freqs)):
        tab = []
        sol,norm_L2 = solver.fem(i)
        tab.append(norm_L2)
        sol,C,norm_L2 = solver.corr_add(i)
        tab.append(norm_L2)
        sol,C,norm_L2 = solver.corr_add_IPP(i)
        tab.append(norm_L2)
        sol,C,norm_L2 = solver.corr_mult(i)
        tab.append(norm_L2)
        tab_errors.append(tab)

    np.save(result_file,tab_errors)
else:
    tab_errors = np.load(result_file+".npy")

In [9]:
columns_name = [method,"Corr_add","Corr_add_IPP","Corr_mult"]
index_name = ["f = "+str(f) for f in freqs]

df_errors = pd.DataFrame(data=tab_errors,columns=columns_name,index=index_name)
df_errors

,fem,Corr_add,Corr_add_IPP,Corr_mult
f = 1,0.002098,2.442384e-10,1.289489e-13,2.968801e-13
f = 2,0.006622,1.533065e-10,1.277896e-13,2.797110e-13
f = 3,0.014114,8.857795e-11,1.273200e-13,2.682653e-13
f = 4,0.024228,9.522978e-11,1.263057e-13,2.609826e-13


In [10]:
df_styled = df_errors.style.format("{:.2e}")
dfi.export(df_styled,result_file+".png",dpi=300)